### Dataset Loading for Food101
This cell defines the `get_data` function, which loads the Food101 dataset using torchvision, applies specified transformations, and creates subsets for training (160 samples per class) and testing (40 samples per class). The function returns the training subset, testing subset, and a list of class names. Ensure the `data_path` directory is valid and writable for downloading/storing the dataset.

In [8]:
from torchvision.datasets import Food101
from torchvision import transforms
from torch.utils.data import Subset
import pathlib
from typing import Tuple, List

def get_data(data_path: pathlib.Path, transform: transforms.Compose) -> Tuple[Subset, Subset, List[str]]:
    """
    Load Food101 dataset and create subsets for training and testing.

    Args:
        data_path (Path): Path to store/load the dataset.
        transform (transforms.Compose): Transformations to apply to the dataset.

    Returns:
        Tuple[Subset, Subset, List[str]]: Training subset, testing subset, and list of class names.
    """
    # Load training data
    food101_train_data = Food101(
        root=data_path / "train",
        split='train',
        transform=transform,
        target_transform=None,
        download=True
    )

    classes = food101_train_data.classes

    # Create training subset (max 160 samples per class)
    class_count = {}
    indices = []
    
    for i, (_, label) in enumerate(food101_train_data):
        if label not in class_count:
            class_count[label] = 0

        if class_count[label] < 160:
            class_count[label] += 1
            indices.append(i)

    train_data = Subset(
        dataset=food101_train_data, 
        indices=indices
    )

    # Load test data
    food101_test_data = Food101(
        root=data_path / "test",
        split='test',
        transform=transform,
        target_transform=None,
        download=True
    )

    # Create test subset (max 40 samples per class)
    class_count = {}
    indices = []

    for i, (_, label) in enumerate(food101_test_data):
        if label not in class_count:
            class_count[label] = 0

        if class_count[label] < 40:
            class_count[label] += 1
            indices.append(i)

    test_data = Subset(
        dataset=food101_test_data,
        indices=indices
    )
        
    return train_data, test_data, classes

### DataLoader Creation for Food101
This cell defines the `get_dataloaders` function, which creates PyTorch DataLoader objects for training and testing subsets of the Food101 dataset. It uses a batch size of 32, disables multiprocessing (`num_workers=0`), enables shuffling for training, and disables it for testing. The function returns DataLoaders for both datasets, optimized with `pin_memory=True` for potential GPU usage.

In [15]:
from torch.utils.data import DataLoader, Subset
from typing import Tuple

def get_dataloaders(train_data: Subset, test_data: Subset) -> Tuple[DataLoader, DataLoader]:
    """
    Create DataLoader objects for training and testing datasets.

    Args:
        train_data (Subset): Training dataset subset.
        test_data (Subset): Testing dataset subset.

    Returns:
        Tuple[DataLoader, DataLoader]: DataLoaders for training and testing datasets.
    """

    # Create dataloader for training dataset
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=32,
        num_workers=0,
        shuffle=True,
        pin_memory=True
    )

    # Create dataloader for testing dataset
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=32,
        num_workers=0,
        shuffle=False,
        pin_memory=True
    )

    return train_dataloader, test_dataloader

### Vision Transformer Model Setup
This cell defines the `get_model` function, which loads a pre-trained Vision Transformer (ViT-B/16) model from torchvision, freezes its parameters, and replaces the classification head with a new linear layer for a specified number of output classes. The function returns the modified model, suitable for transfer learning. Ensure `out_features` matches the number of dataset classes (e.g., 101 for Food101).

In [16]:
from torchvision.models import ViT_B_16_Weights, vit_b_16
from torch import nn

def get_model(out_features: int) -> nn.Module:
    """
    Create a Vision Transformer (ViT-B/16) model with a modified classification head.

    Args:
        out_features (int): Number of output classes for the classification head.

    Returns:
        nn.Module: Modified ViT-B/16 model with frozen parameters and a new classification head.
    """
    # Load pre-trained ViT-B/16 model
    model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)

    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Replace the classification head
    model.heads.head = nn.Linear(
        in_features=768,
        out_features=out_features,
        bias=True
    )

    return model

### ViT-B/16 Image Transformations
This cell defines the `get_transform` function, which retrieves the default image transformation pipeline for the pre-trained Vision Transformer (ViT-B/16) model from torchvision. The transformations include resizing, center cropping, tensor conversion, and normalization suitable for ViT-B/16. The function returns a `transforms.Compose` object.

In [17]:
from torchvision.models import ViT_B_16_Weights
from torchvision import transforms

def get_transform() -> transforms.Compose:
    """
    Retrieve the default image transformations for the pre-trained ViT-B/16 model.

    Returns:
        transforms.Compose: A composed transform pipeline for ViT-B/16.
    """
    return ViT_B_16_Weights.DEFAULT.transforms()

### Model Training and Evaluation
This cell defines the `train_set` and `test_set` functions for training and evaluating a PyTorch model on a dataset. The `train_set` function runs one epoch of training, updating model parameters using the specified optimizer and loss function. The `test_set` function evaluates the model without gradient computation. Both functions calculate average loss and accuracy (in percentage) on the given device (e.g., CPU or GPU). Ensure the model, DataLoader, loss function, and device are compatible with the target dataset.

In [19]:
from sklearn.metrics import accuracy_score
import torch
from typing import Tuple

def train_set(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module, 
              optimizer: torch.optim.Optimizer, 
              device: torch.device) -> Tuple[float, float]:
    """
    Train the model for one epoch on the training dataset.

    Args:
        model (torch.nn.Module): The neural network model to train.
        dataloader (torch.utils.data.DataLoader): DataLoader for the training dataset.
        loss_fn (torch.nn.Module): Loss function for training.
        optimizer (torch.optim.Optimizer): Optimizer for updating model parameters.
        device (torch.device): Device to run the model on (e.g., 'cuda' or 'cpu').

    Returns:
        Tuple[float, float]: Average training loss and accuracy (in percentage) for the epoch.
    """
    model.train()
    train_loss = 0.0
    accuracy = 0.0

    for X, y in dataloader:
        # Move data to device
        X, y = X.to(device), y.to(device)
        
        # Forward pass
        y_logits = model(X)
        loss = loss_fn(y_logits, y)
        
        # Compute predictions and accuracy
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
        # Move to CPU and convert to NumPy for sklearn
        y_cpu, y_pred_cpu = y.cpu().detach().numpy(), y_pred.cpu().detach().numpy()
        accuracy += accuracy_score(y_cpu, y_pred_cpu)

        train_loss += loss.item()
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Compute average loss and accuracy
    train_loss /= len(dataloader)
    accuracy = (accuracy / len(dataloader)) * 100

    return train_loss, accuracy


def test_set(model: torch.nn.Module, 
             dataloader: torch.utils.data.DataLoader, 
             loss_fn: torch.nn.Module, 
             device: torch.device) -> Tuple[float, float]:
    """
    Evaluate the model on the test dataset.

    Args:
        model (torch.nn.Module): The neural network model to evaluate.
        dataloader (torch.utils.data.DataLoader): DataLoader for the test dataset.
        loss_fn (torch.nn.Module): Loss function for evaluation.
        device (torch.device): Device to run the model on (e.g., 'cuda' or 'cpu').

    Returns:
        Tuple[float, float]: Average test loss and accuracy (in percentage).
    """
    model.eval()
    test_loss = 0.0
    accuracy = 0.0

    with torch.inference_mode():
        for X, y in dataloader:
            # Move data to device
            X, y = X.to(device), y.to(device)
            
            # Forward pass
            y_logits = model(X)
            loss = loss_fn(y_logits, y)
            
            # Compute predictions and accuracy
            y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
            # Move to CPU and convert to NumPy for sklearn
            y_cpu, y_pred_cpu = y.cpu().detach().numpy(), y_pred.cpu().detach().numpy()
            accuracy += accuracy_score(y_cpu, y_pred_cpu)
            
            test_loss += loss.item()

    # Compute average loss and accuracy
    test_loss /= len(dataloader)
    accuracy = (accuracy / len(dataloader)) * 100

    return test_loss, accuracy

### Function for engine

In [20]:
import torch
from typing import Dict, List

def engine(model: torch.nn.Module, 
           train_dataloader: torch.utils.data.DataLoader,
           test_dataloader: torch.utils.data.DataLoader, 
           loss_fn: torch.nn.Module, 
           optimizer: torch.optim.Optimizer, 
           device: torch.device, 
           epochs: int) -> Dict[str, List[float]]:
    """
    Train and evaluate a PyTorch model over multiple epochs, tracking performance metrics.

    Args:
        model (torch.nn.Module): The neural network model to train and evaluate.
        train_dataloader (torch.utils.data.DataLoader): DataLoader for the training dataset.
        test_dataloader (torch.utils.data.DataLoader): DataLoader for the test dataset.
        loss_fn (torch.nn.Module): Loss function for training and evaluation.
        optimizer (torch.optim.Optimizer): Optimizer for updating model parameters.
        device (torch.device): Device to run the model on (e.g., 'cuda' or 'cpu').
        epochs (int): Number of training epochs.

    Returns:
        Dict[str, List[float]]: Dictionary containing lists of epoch numbers, training losses,
                                training accuracies, test losses, and test accuracies.
    """
    # Initialize results dictionary
    result = {
        'epoch': [],
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }

    for i in range(epochs):
        # Train for one epoch
        train_loss, train_acc = train_set(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        # Evaluate on test set
        test_loss, test_acc = test_set(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device
        )

        # Store metrics
        result['epoch'].append(i + 1)
        result['train_loss'].append(train_loss)
        result['train_acc'].append(train_acc)
        result['test_loss'].append(test_loss)
        result['test_acc'].append(test_acc)

        # Print progress
        print(f"Epoch {i + 1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

    return result

### Function for save model

In [29]:
import torch
from pathlib import Path

def save_model(path: Path, model: torch.nn.Module) -> None:
    """
    Save the model's state dictionary to a specified file path.

    Args:
        path (Path): Directory path where the model will be saved.
        model (torch.nn.Module): The PyTorch model to save.

    Returns:
        None: The function saves the model and prints a confirmation message.
    """
    # Ensure the directory exists
    path.mkdir(parents=True, exist_ok=True)
    
    # Define the file path
    file_name = path / "vit_model_on_food101.pth"
    
    # Save the model's state dictionary
    torch.save(obj=model.state_dict(), f=file_name)
    
    # Print confirmation
    print(f"The model has been saved successfully to {file_name}")